In [4]:
import sys
sys.path.append('../')

In [5]:
import tensorflow as tf

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

class_names = ['airplane', 'automobile', 'bird', 'cat',
               'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

def preprocess_data(image, label):
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0
    return image, label

# Convert to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

# Resize and normalize images on-the-fly
train_dataset = train_dataset.map(preprocess_data, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(preprocess_data, num_parallel_calls=tf.data.AUTOTUNE)

# Batch, shuffle, and prefetch
BATCH_SIZE = 32
train_dataset = train_dataset.shuffle(50000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [6]:
import pickle

base_model = tf.keras.applications.MobileNetV3Small(
    weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

epochs = 100
batch_size = 64
dropout = .25

model = tf.keras.Sequential([
    base_model, tf.keras.layers.GlobalAveragePooling2D(), 
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(dropout),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(dropout),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(dropout),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_dataset, epochs=epochs, batch_size=batch_size, validation_data=(test_dataset), callbacks=tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=2))

model.save('model.keras')
model.save_weights('weights.keras')

with open('history.pkl', 'wb') as file:
    pickle.dump(history.history, file)

with open('history.pkl', 'rb') as file:
    history = pickle.load(file)

loss, accuracy = model.evaluate(test_images, test_labels)

print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

Epoch 1/100
 181/1563 [==>...........................] - ETA: 7:31 - loss: 2.3103 - accuracy: 0.1020

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, cohen_kappa_score
import numpy as np
from plotting import roc_plot, plot_history, cm_plot

plot_history(history)

y_prob = model.predict(test_images, verbose=2)
y_pred = np.argmax(y_prob, axis=1)

roc_plot(test_labels, y_prob, class_names)
print(cohen_kappa_score(test_labels.squeeze(), y_pred))
print(classification_report(test_labels, y_pred, target_names=class_names))
cm_plot(confusion_matrix(test_labels, y_pred), class_names)